In [1]:
import numpy as np
dataset_filename="/Users/liuhuanshuo/Desktop/Python_book/Python数据挖掘入门与实战/Code_REWRITE/Chapter 1/affinity_dataset.txt"
#加载数据文件
dataset=np.loadtxt(dataset_filename)

In [18]:
print(dataset[:5]) #一行表示一个用户对五个商品的购买情况,1表示买了，0表示没买


[[1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [19]:
n_samples, n_features=dataset.shape
print("{0} samples in total with {1} features".format(n_samples,n_features))


100 samples in total with 5 features


In [21]:
#定义特征名集合
features=["bread","milk","cheese","apple","banana"]

In [22]:
#目标一 ， 找出以下规则:要找出用户购买了apple, 那他也可能购买banana
# 前提：计算规则的支持度和置信度
#支持度:数据集中规则应验的次数.
#置信度: 符合给定条件的所有规则里，跟当前规则结论一致的比例有多少. 


In [23]:
#首先, 有多少行有这条规则的前提条件
num_apple_purchases=0
for sample in dataset:
    if sample[3]==1:
        num_apple_purchases+=1
print("{0} people bought apples".format(num_apple_purchases))

38 people bought apples


In [24]:
#下面可以计算这条规则的应验数和无效数
rule_valid=0.0
rule_invalid=0
for sample in dataset:
    if sample[3]==1:
        if sample[4]==1:
            rule_valid+=1
        else:
            rule_invalid+=1
print("{0} cases of the rule being valid".format(rule_valid))
print("{0} cases of the rule being invalid".format(rule_invalid))

21.0 cases of the rule being valid
17 cases of the rule being invalid


In [25]:
#支持度
support=rule_valid
#置信度
confidence=rule_valid/num_apple_purchases
print("the support is {0} and the confidence is {1:.3f}".format(support, confidence))

the support is 21.0 and the confidence is 0.553


In [26]:
################################################################
#有了上面的案例，用于理解支持度和置信度
#下面开始所有的规则的支持度和置信度的计算

In [27]:
from collections import defaultdict
#数据格式采用字典：{ 键(前提,结论), 值(数值)}
valid_rules=defaultdict(int)    #每条规则的支持度
invalid_rules=defaultdict(int)  #每条规则的无效数
#数据格式采用字典：{ 键(前提), 值(数值)}
num_rule_occurances=defaultdict(float)   #规则条件出现的次数


In [28]:
n_features

5

In [29]:
for sample in dataset:
    for premise in range(n_features):    
        if sample[premise] ==0: continue    #这个商品没有购买，则表示这个前提不成立,跳过循环
        num_rule_occurances[premise]+=1.0     #出现了一次前提条件
        for conclusion in range(n_features):   #计算它的支持度和无效数
            if premise==conclusion:
                continue
            if sample[conclusion]==1:
                valid_rules[(premise,conclusion)]+=1
            else:
                invalid_rules[(premise,conclusion)]+=1
                              
support=valid_rules
confidence=defaultdict(float)
for premise,conclusion in valid_rules.keys():
    confidence[(premise,conclusion)]=valid_rules[(premise,conclusion)]/num_rule_occurances[premise]

In [30]:
#下面将结果输出一下
for premise, conclusion in confidence:
    #求出规则前提条件商品名
    premise_name=features[premise]
    #求出规则条件的结论对应的商品名
    conclusion_name=features[conclusion]
    print("Rule: if a person buys {0} they will also buy {1}".format(premise_name,conclusion_name))
    print(" this rule's support is :{0}".format( support[(premise,conclusion)]     )  )
    print(" this rule's confidence is :{0}".format( confidence[(premise,conclusion)]     )  )
    print("")
    

Rule: if a person buys bread they will also buy banana
 this rule's support is :13
 this rule's confidence is :0.52

Rule: if a person buys banana they will also buy bread
 this rule's support is :13
 this rule's confidence is :0.21666666666666667

Rule: if a person buys bread they will also buy milk
 this rule's support is :15
 this rule's confidence is :0.6

Rule: if a person buys milk they will also buy bread
 this rule's support is :15
 this rule's confidence is :0.2631578947368421

Rule: if a person buys milk they will also buy banana
 this rule's support is :32
 this rule's confidence is :0.5614035087719298

Rule: if a person buys banana they will also buy milk
 this rule's support is :32
 this rule's confidence is :0.5333333333333333

Rule: if a person buys cheese they will also buy apple
 this rule's support is :21
 this rule's confidence is :0.5

Rule: if a person buys cheese they will also buy banana
 this rule's support is :27
 this rule's confidence is :0.6428571428571429



In [31]:
'''
  将输出部分定义成一个函数
  premise: 前提 的索引
  conclusion: 结论的索引
  support: 支持度字典
  confidence: 置信度字典
  features: 特征数组
'''
def print_rule( premise,conclusion,support,confidence,features):
     #求出规则前提条件商品名
    premise_name=features[premise]
    #求出规则条件的结论对应的商品名
    conclusion_name=features[conclusion]
    print("Rule: if a person buys {0} they will also buy {1}".format(premise_name,conclusion_name))
    print(" this rule's support is :{0}".format( support[(premise,conclusion)]     )  )
    print(" this rule's confidence is :{0}".format( confidence[(premise,conclusion)]     )  )
    print("")

In [32]:
    
#调用上面的函数，输出计算好的规则的支持度和置信度,比如，买了milk后又买apple的支持度和置信度
premise=1
conclusion=3
print_rule(premise,conclusion,support,confidence,features)

Rule: if a person buys milk they will also buy apple
 this rule's support is :17
 this rule's confidence is :0.2982456140350877



In [33]:
###################################################################################
'''
  下面对支持度进行排序
'''
#先排序输出
from pprint import pprint
pprint(list(support.items()))
#排好序后存到字典中
from operator import itemgetter
#                根据支持度的值排序 
sorted_support=sorted(support.items(), key=itemgetter(1),reverse=True)
#输出前三名
for index in range(5):
    print("Rule #{0}".format(index+1))
    (premise,conclusion)=sorted_support[index][0]   #取出键
    print_rule( premise, conclusion, support,confidence,features)

[((0, 4), 13),
 ((4, 0), 13),
 ((0, 1), 15),
 ((1, 0), 15),
 ((1, 4), 32),
 ((4, 1), 32),
 ((2, 3), 21),
 ((2, 4), 27),
 ((3, 2), 21),
 ((3, 4), 21),
 ((4, 2), 27),
 ((4, 3), 21),
 ((0, 2), 3),
 ((1, 2), 12),
 ((2, 0), 3),
 ((2, 1), 12),
 ((1, 3), 17),
 ((3, 1), 17),
 ((0, 3), 5),
 ((3, 0), 5)]
Rule #1
Rule: if a person buys milk they will also buy banana
 this rule's support is :32
 this rule's confidence is :0.5614035087719298

Rule #2
Rule: if a person buys banana they will also buy milk
 this rule's support is :32
 this rule's confidence is :0.5333333333333333

Rule #3
Rule: if a person buys cheese they will also buy banana
 this rule's support is :27
 this rule's confidence is :0.6428571428571429

Rule #4
Rule: if a person buys banana they will also buy cheese
 this rule's support is :27
 this rule's confidence is :0.45

Rule #5
Rule: if a person buys cheese they will also buy apple
 this rule's support is :21
 this rule's confidence is :0.5



In [34]:
'''
  下面对信任度进行排序
'''
#先排序输出
pprint(list(confidence.items()))
#排好序后存到字典中
#                根据信任度的值排序 
sorted_confidence=sorted(confidence.items(), key=itemgetter(1),reverse=True)
#输出前三名
for index in range(5):
    print("Rule #{0}".format(index+1))
    (premise,conclusion)=sorted_confidence[index][0]   #取出键
    print_rule( premise, conclusion, support,confidence,features)

[((0, 4), 0.52),
 ((4, 0), 0.21666666666666667),
 ((0, 1), 0.6),
 ((1, 0), 0.2631578947368421),
 ((1, 4), 0.5614035087719298),
 ((4, 1), 0.5333333333333333),
 ((2, 3), 0.5),
 ((2, 4), 0.6428571428571429),
 ((3, 2), 0.5526315789473685),
 ((3, 4), 0.5526315789473685),
 ((4, 2), 0.45),
 ((4, 3), 0.35),
 ((0, 2), 0.12),
 ((1, 2), 0.21052631578947367),
 ((2, 0), 0.07142857142857142),
 ((2, 1), 0.2857142857142857),
 ((1, 3), 0.2982456140350877),
 ((3, 1), 0.4473684210526316),
 ((0, 3), 0.2),
 ((3, 0), 0.13157894736842105)]
Rule #1
Rule: if a person buys cheese they will also buy banana
 this rule's support is :27
 this rule's confidence is :0.6428571428571429

Rule #2
Rule: if a person buys bread they will also buy milk
 this rule's support is :15
 this rule's confidence is :0.6

Rule #3
Rule: if a person buys milk they will also buy banana
 this rule's support is :32
 this rule's confidence is :0.5614035087719298

Rule #4
Rule: if a person buys apple they will also buy cheese
 this rule's s